In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np

from tensorflow import keras
from tensorflow.keras import layers

from sklearn.utils import shuffle

In [8]:
# process data
from sklearn.utils import shuffle
    
def column_mean_and_std_fn(df, col):
    not_nan = df[col].notna()
    tmp_df = df[col][not_nan]
    return  tmp_df.mean(), tmp_df.std()

def column_fillna(df, col, value):
    tmp_df = df[col].fillna(value)
    return tmp_df 

# 构造特征字典
def gen_feat_dict(df,ignore_cols=[] ):
    feat_dict = {}
    tc = 1
    for col in df.columns:
        if col in df_categorical_columns:
            us = df[col].unique()
            feat_dict[col] = dict(zip(us, range(tc, len(us)+tc)))
            tc += len(us)
    return tc, feat_dict

# 对某一列去掉低频特征
def remove_rare_value(x, args):
    _set = args[0]
    col = args[1]
    if x not in _set:
        x = "%s_remove"%col
        return x
    else:
        return x
        
    

In [9]:
df = pd.read_csv("./criteo_sampled_data.csv")
df = shuffle(df)

# 去掉一些取值很分散的特征
omit_columns = ['C3', 'C9', 'C12', 'C16', 'C21']
df = df.drop(omit_columns, axis=1)

# 剩下的特征，按照numerci和categorical分开
df_numeric_columns = ['I1', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11', 'I12', 'I13']
df_categorical_columns = ['C1', 'C2', 'C4', 'C5', 'C6', 'C7', 'C8', 'C10', 'C11', 
                        'C13', 'C14', 'C15', 'C17', 'C18','C19', 'C20',
                       'C22', 'C23', 'C24', 'C25', 'C26', 'I2']

# 填充缺失值
for col in df_numeric_columns:
    mean, std = column_mean_and_std_fn(df, col)
    df[col] = column_fillna(df, col, mean)
    df[col] = (df[col]-mean)/std
for col in df_categorical_columns:
    value = ("%s_nan"%(col))
    df[col] = df[col].fillna(value)

# 对剩下的categorical还要去掉一些频次很低的特征
categorical_columns_dict = {}
for col in df_categorical_columns[0:]:
    tmp = df.groupby([col]).size().sort_values(ascending=False).cumsum() / 600000
    feature_list = tmp[tmp<=0.80]
    categorical_columns_dict[col] = set(feature_list.index.tolist())

In [10]:
# 将低频特征替换为 "col_remove" 的字符串
for col in df_categorical_columns:
    _set = categorical_columns_dict[col]
    df[col] = df[col].apply(remove_rare_value, args=((_set, col),))

In [11]:
labels = df.pop("label")
features = df

print(features)
print(labels)

ft_sz, ft_dict = gen_feat_dict(features)


              I1         I2        I3        I4         I5        I6  \
476485  0.000000         -1  0.000000  0.000000  10.307109  0.000000   
593808  0.074609        180 -0.052305 -0.675870  -0.275100 -0.236530   
103361  0.000000         46 -0.040468  0.008152  -0.063902 -0.220677   
375145 -0.139874          0  0.000000  0.000000  -0.265967 -0.208786   
368254  0.000000  I2_remove -0.049937 -0.105852  -0.111832 -0.139426   
...          ...        ...       ...       ...        ...       ...   
42937   0.000000          2 -0.052305 -0.561866  -0.231340 -0.234549   
256072 -0.354356          2 -0.054672 -0.447863  -0.171758  0.000000   
307706 -0.354356          3  0.000000  0.000000  -0.235944  0.019112   
289958 -0.354356         22  0.042393 -0.675870   0.150040  0.000000   
336620 -0.354356          1 -0.035733 -0.333859  -0.253895 -0.157262   

              I7        I8        I9       I10  ...         C15         C17  \
476485  0.000000 -0.261545  0.000000  0.000000  ...    4

In [12]:
print(ft_sz)

13945


In [13]:
print(type(ft_dict))
print(ft_dict.keys())


<class 'dict'>
dict_keys(['I2', 'C1', 'C2', 'C4', 'C5', 'C6', 'C7', 'C8', 'C10', 'C11', 'C13', 'C14', 'C15', 'C17', 'C18', 'C19', 'C20', 'C22', 'C23', 'C24', 'C25', 'C26'])


In [14]:
889455*(17)*64 / 1024 / 1024 / 1024
import json
def value2index_func(x, args):
    tmp_dict = args
    if x in tmp_dict:
        return tmp_dict[x]
    else:
        return 0
    

for col in df_categorical_columns:
    _dict = ft_dict[col]
    df[col] = df[col].apply(value2index_func,args=(_dict,))

print(df)

              I1  I2        I3        I4         I5        I6        I7  \
476485  0.000000   1  0.000000  0.000000  10.307109  0.000000  0.000000   
593808  0.074609   2 -0.052305 -0.675870  -0.275100 -0.236530  0.931599   
103361  0.000000   3 -0.040468  0.008152  -0.063902 -0.220677 -0.236709   
375145 -0.139874   4  0.000000  0.000000  -0.265967 -0.208786 -0.203329   
368254  0.000000   5 -0.049937 -0.105852  -0.111832 -0.139426 -0.169949   
...          ...  ..       ...       ...        ...       ...       ...   
42937   0.000000  18 -0.052305 -0.561866  -0.231340 -0.234549 -0.236709   
256072 -0.354356  18 -0.054672 -0.447863  -0.171758  0.000000 -0.253399   
307706 -0.354356   7  0.000000  0.000000  -0.235944  0.019112 -0.236709   
289958 -0.354356  15  0.042393 -0.675870   0.150040  0.000000  0.000000   
336620 -0.354356   6 -0.035733 -0.333859  -0.253895 -0.157262 -0.203329   

              I8        I9       I10  ...    C15    C17    C18    C19    C20  \
476485 -0.261545  0

In [15]:
print(labels)

476485    0
593808    0
103361    0
375145    0
368254    1
         ..
42937     0
256072    0
307706    0
289958    0
336620    1
Name: label, Length: 600000, dtype: int64


In [16]:
df.to_csv("features.csv",index=False)
labels.to_csv("labels.csv", index=False)

In [17]:
def foo(*args, **kwargs):
    print('args = ', args)
    print('kwargs = ', kwargs)
    for k in kwargs:
        print(k)
        print(kwargs[k])
    
#     b = json.loads(args(0))
#     print(b)
    
    print('---------------------------------------')

In [18]:
import json
a = ({"a":1,"b":2})
print(a)
foo(a=a)

{'a': 1, 'b': 2}
args =  ()
kwargs =  {'a': {'a': 1, 'b': 2}}
a
{'a': 1, 'b': 2}
---------------------------------------
